<a href="https://colab.research.google.com/github/raquelcarmo/tropical_cyclones/blob/main/src/code/TC_model_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Comparison
Script to execute performance comparison between trained models.

## Imports and configurations

In [ ]:
# Insert your desired path to work on
import os
os.chdir('../data')

In [ ]:
# General imports
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pickle
import scipy
import seaborn as sns
import itertools

## 1. Comparison between X models in one dataset

In [ ]:
main_dir = "VV_VH_WS_dilated"
sub_dir = f"{main_dir}/results/categorization/test_eye_only"

# note that the result (in this case the recall) could be null if the classifier is broken 
# (all recall = 1 because is always classified as positive class)
MODEL_1_DIRECTORY = f"{sub_dir}/ResNet_nu-F_bs-8_416x416_lr-1e-05_ep-30_sp-5_no-zT_cr-u1_ag-T_drp-F/csv"
MODEL_2_DIRECTORY = f"{sub_dir}/test_new_weights/ResNet_nu-F_bs-8_416x416_lr-1e-05_ep-30_sp-5_no-zT_cr-u1_ag-T_drp-F/csv"
#MODEL_3_DIRECTORY = f"{sub_dir}/test_new_weights/ResNet_nu-F_bs-8_416x416_lr-1e-05_ep-30_sp-5_no-zT_cr-u1_ag-T_drp-F/csv"
#MODEL_4_DIRECTORY = f"{sub_dir}/test_rmse/ResNet_nu-F_bs-8_416x416_lr-1e-05_ep-30_sp-5_no-mT_cr-u1_ag-T_drp-F/csv"
#MODEL_5_DIRECTORY = f"{sub_dir}/ResNet_nu-F_bs-8_416x416_lr-0.0001_ep-35_sp-5_no-zT_cr-u1_ag-T_drp-F_ft-last50/csv"
#MODEL_6_DIRECTORY = f"{sub_dir}/ResNet_nu-F_bs-8_416x416_lr-0.0001_ep-35_sp-5_no-zT_cr-u1_ag-T_drp-F_ft-last75/csv"
#MODEL_7_DIRECTORY = f"{sub_dir}/ResNet_nu-F_bs-8_416x416_lr-0.0001_ep-35_sp-5_no-zT_cr-u1_ag-T_drp-F_ft-last125/csv"
#MODEL_8_DIRECTORY = f"{sub_dir}/ResNet_nu-F_bs-8_416x416_lr-0.0001_ep-35_sp-5_no-zT_cr-u1_ag-T_drp-F_ft-last175/csv"
#MODEL_9_DIRECTORY = f"{sub_dir}/ResNet_nu-F_bs-8_416x416_lr-1e-05_ep-30_sp-5_no-zT_cr-u1_ag-T_drp-F/csv"

#dir_dict = {1: MODEL_1_DIRECTORY, 2: MODEL_2_DIRECTORY, 3: MODEL_3_DIRECTORY,\
#            4: MODEL_4_DIRECTORY, 5: MODEL_5_DIRECTORY, 6: MODEL_6_DIRECTORY,\
#            7: MODEL_7_DIRECTORY, 8: MODEL_8_DIRECTORY, 9: MODEL_9_DIRECTORY}
#dir_dict = {1: MODEL_1_DIRECTORY, 2: MODEL_2_DIRECTORY, 3: MODEL_3_DIRECTORY, 4: MODEL_4_DIRECTORY, 5: MODEL_5_DIRECTORY}
#dir_dict = {1: MODEL_1_DIRECTORY, 2: MODEL_2_DIRECTORY, 3: MODEL_3_DIRECTORY, 4: MODEL_4_DIRECTORY}
#dir_dict = {1: MODEL_1_DIRECTORY, 2: MODEL_2_DIRECTORY, 3: MODEL_3_DIRECTORY}
dir_dict = {1: MODEL_1_DIRECTORY, 2: MODEL_2_DIRECTORY}

# Settings
models = len(dir_dict)
save_plot = True
#title = "Comparison between dropout rates - ResNet_nu-F_bs-8_384x384_lr-1e-05_ep-30_sp-5_no-mT_cr-u1_ag-T_drp-XXX"
#title = "Comparison between input sizes - ResNet_nu-F_bs-8_XXX_lr-0.0001_ep-20_sp-5_no-zT_cr-u1_ag-T"
#title = "Comparison between normalizations" - ResNet_nu-F_bs-8_384x384_lr-1e-05_ep-30_sp-5_no-XXX_cr-u1_ag-T_drp-F"
#title = "Comparison between nb crops"# - ResNet_nu-F_bs-8_XXX_lr-1e-05_ep-30_sp-5_no-mT_cr-uXXX_ag-T_drp-F"
#title = "Comparison between nb epochs - ResNet_nu-F_bs-8_288x288_lr-0.0001_ep-XXX_sp-5_no-mT_cr-u1_ag-T_ft-165"
#title = "Comparison between learning rates - ResNet_nu-F_bs-8_384x384_lr-XXX_ep-XXX_sp-5_no-zT_cr-u1_ag-T"
#title = "Comparison between nb frozen layers - ResNet_nu-F_bs-8_416x416_lr-XXX_ep-XXX_sp-5_no-zT_cr-u1_ag-T_drp-F_ft-XXX"
#title = "Comparison between CNNs - XXX_nu-F_bs-8_288x288_lr-0.0001_ep-20_sp-5_no-mT_cr-u1_ag-T"
#title = "Comparison between nb of classes - ResNet_nu-F_bs-8_416x416_lr-XXX_ep-XXX_sp-5_no-XXXT_cr-u1_ag-T"
#title = "Comparison between rotations - Model 1 has two more augmentation techniques (rot180 and rot270)"
#title = 'Comparison between zeroed land (Model 1) and land (Model 2) - ResNet_nu-F_bs-8_416x416_lr-1e-05_ep-30_sp-5_no-mT_cr-u1_ag-T_drp-F'
#title = 'CrossEntropy Loss (Models 1 and 2) and Combined CrossEntropy and RMSE Loss (Models 3 and 4)'
title = 'Normal Class weights (Model 1) and Class weights with prior precision (Model 2) - ResNet_nu-F_bs-8_416x416_lr-1e-05_ep-30_sp-5_no-zT_cr-u1_ag-T_drp-F'
file_name = title.replace(" ", "_")

# Critical t-value to see the difference in two distribution with 5 samples
criticalTvalue = 1.533      # 80% confidence
#criticalTvalue = 2.132     # 90% confidence
#criticalTvalue = 2.776     # 95% confidence

### 1.1. Load necessary functions

In [ ]:
def mean_confidence_interval(data, confidence=0.80):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h


def compute_f1(dir):
    precision = np.asarray(pickle.load(open(f"{dir}/test_precision.pkl", "rb")))
    recall = np.asarray(pickle.load(open(f"{dir}/test_recall.pkl", "rb")))
    f1 = 2 * (precision * recall) / (precision + recall)
    f1 = np.nan_to_num(f1)
    with open(f"{dir}/test_f1-score.pkl", 'wb') as handle:
        pickle.dump(f1.tolist(), handle, protocol=pickle.HIGHEST_PROTOCOL)


def compute_stats(nb_models, dir_dict, metrics):
    inter_dict = {}
    mean_dict = {}
    for metric in metrics:
        metric_dict = {}
        for i in range(1, nb_models+1):
            load_metric = pickle.load(open(f"{dir_dict[i]}/test_{metric}.pkl", "rb"))
            metric_dict['model%d_%s' %(i, metric)] = load_metric

            mean, b, u = mean_confidence_interval(load_metric)
            #print(mean, b, u)
            #conf_dict['model%d_%s' %(i, metric)] = [mean, b, u]

            if 'model%d' %(i) in inter_dict:
                inter_dict['model%d' %(i)].append(u - mean)
                mean_dict['model%d' %(i)].append(mean)
            else:
                inter_dict['model%d' %(i)] = [u - mean]
                mean_dict['model%d' %(i)] = [mean]

        #print(inter_dict)
        #print(mean_dict)
        print(f"---------------- {metric.upper()} analysis ----------------")
        for x, y in itertools.combinations(metric_dict.keys(), 2):
            t, p = scipy.stats.ttest_ind(metric_dict[x], metric_dict[y])
            print(f"t value {metric} between {x} and {y}: {t}")
            if t > criticalTvalue or t < -criticalTvalue :
                print(f"Null hypothesis rejected -> there is a significant difference in the two {metric} distributions.")
            else:
                print(f"Null hypothesis not rejected -> I assume that each difference in {metric} is due to the randomness.")
        print("-------------------------------------------------")
    return mean_dict, inter_dict


def plot_model_comparisons(nb_models, metrics, dir_dict, mean_dict, inter_dict, sub_dir, title, save_plot):
    # Get colors from palette
    colors = sns.color_palette("hls", nb_models)

    fig, ax = plt.subplots(figsize=(15,8))
    index = np.arange(len(metrics))
    bar_width = 0.1
    opacity = 0.8

    for i in range(1, nb_models+1):
        rects = plt.bar(index + (i-1)*bar_width, mean_dict['model%d' %(i)], bar_width, yerr = inter_dict['model%d' %(i)],
                        capsize = 5, alpha = opacity, color = colors[i-1],
                        label = dir_dict[i].split(sep='/')[-2])

    font_size = 15
    font_size_ticks = 13
    plt.ylabel('Scores', fontsize = font_size)
    plt.title(title, fontsize = font_size)
    plt.xticks(index + (nb_models/2-0.5)*bar_width, tuple([name.capitalize() for name in metrics]), fontsize = font_size)
    plt.yticks(fontsize = font_size_ticks)
    #plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True, shadow=True, ncol=2, prop={'size': font_size-3})
    plt.legend(loc = 'lower left', prop={'size': font_size})
    plt.grid(True)
    #ax.set_ylim(ymin=0.75)
    plt.tight_layout()
    plt.show()

    # Save plot
    if save_plot:
        dir = f"{sub_dir}/model_comparisons"
        os.makedirs(dir, exist_ok=True)
        fig.savefig("{}/{}.jpg".format(dir, title.replace(" ", "_")), bbox_inches='tight')


def plot_frozen_layers(nb_models, metrics, dir_dict, mean_dict, inter_dict, sub_dir, title, save_plot):
    # Get colors from palette
    colors = sns.color_palette("hls", nb_models)
    a = np.arange(nb_models)
    x = [0, 165, 170, 172, 174]
    assert nb_models == len(x)

    fig, ax = plt.subplots(figsize=(15,8))

    for j in range(len(metrics)):
        metric_mean = []
        metric_yerr = []
        for i in range(1, nb_models+1):
            #ax.plot(a, mean_dict['model%d' %(i)], color = colors[i-1], label = dir_dict[i].split(sep='/')[-2])
            metric_mean.append(mean_dict['model%d' %(i)][j])
            metric_yerr.append(inter_dict['model%d' %(i)][j])

        ax.errorbar(a, metric_mean, yerr = metric_yerr, capsize=5, capthick=2,
                    fmt='-o', color = colors[j], label = metrics[j].capitalize())
        bellow = np.asarray(metric_mean)-np.asarray(metric_yerr)
        up = np.asarray(metric_mean)+np.asarray(metric_yerr)
        ax.fill_between(a, bellow.tolist(), up.tolist(), color=colors[j], alpha=.1)
    
    font_size = 15
    font_size_ticks = 13
    plt.xlabel('Number of frozen layers in backbone', fontsize = font_size)
    plt.ylabel('Scores', fontsize = font_size)
    plt.title(title, fontsize = font_size)
    ax.xaxis.set_ticks(a) #set the ticks to be a
    ax.xaxis.set_ticklabels(x, fontsize = font_size_ticks) # change the ticks' names to x
    #ax.set_ylim(ymin=0.25)
    #plt.xticks(x, fontsize = font_size_ticks)
    start, end = ax.get_ylim()
    ax.yaxis.set_ticks(np.arange(np.round(start, 1), np.round(end, 1), 0.05))
    plt.yticks(fontsize = font_size_ticks)
    plt.legend(loc = 'lower left', prop={'size': font_size})
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Save plot
    if save_plot:
        dir = f"{sub_dir}/model_comparisons"
        os.makedirs(dir, exist_ok=True)
        fig.savefig("{}/{}.jpg".format(dir, title.replace(" ", "_")), bbox_inches='tight')

### 1.2. Metric analysis and plotting

In [ ]:
dirs = ["VV_VH_WS/results/identification/ResNet_Numeric-F_BatchSize-8_lr-0.0001_Epochs-20_Folds-5_Norm-T/csv", 
        "WS_WS_WS/results/identification/ResNet_Numeric-F_BatchSize-8_lr-0.0001_Epochs-20_Folds-5_Norm-T/csv",
        "VV_VH_VH/results/identification/ResNet_Numeric-F_BatchSize-8_lr-0.0001_Epochs-20_Folds-5_Norm-T/csv",
        "WS_sWSO_cWSO/results/identification/ResNet_Numeric-F_BatchSize-8_lr-0.0001_Epochs-20_Folds-5_Norm-T/csv"]

for check_dir in dirs:
    print(check_dir.split('/')[0])
    file = glob(f'{check_dir}/test_f1-score.pkl')

    if file == []:
        compute_f1(check_dir)
    #check_dir = dir_dict[9]

    acc = np.asarray(pickle.load(open(f"{check_dir}/test_accuracy.pkl", "rb")))
    prec = np.asarray(pickle.load(open(f"{check_dir}/test_precision.pkl", "rb")))
    rec = np.asarray(pickle.load(open(f"{check_dir}/test_recall.pkl", "rb")))
    f1 = np.asarray(pickle.load(open(f"{check_dir}/test_f1-score.pkl", "rb")))
    
    print("Accuracy:", acc, np.mean(acc), np.std(acc))
    print("Precision:", prec, np.mean(prec), np.std(prec))
    print("Recall:", rec, np.mean(rec), np.std(rec))
    print("F1-score:", f1, np.mean(f1), np.std(f1))
    print('\n')

In [ ]:
# Compute F1-score if not already computed previously
for i in range(1, models+1):
    dir = dir_dict[i]
    file = glob(f'{dir}/test_f1-score.pkl')
    if file == []:
        compute_f1(dir)

# Metrics to use for analysis
metrics = ["accuracy", "precision", "recall", "f1-score"]

# Compute mean and confidence intervals for each metric and each model
mean_dict, inter_dict = compute_stats(models, dir_dict, metrics)

# Plot model comparison
plot_model_comparisons(models, metrics, dir_dict, mean_dict, inter_dict, sub_dir, title, save_plot)

### 1.3. Plot Scores vs. Number of frozen layers

In [ ]:
# Compute F1-score if not already computed previously
for i in range(1, models+1):
    dir = dir_dict[i]
    file = glob(f'{dir}/test_f1-score.pkl')
    if file == []:
        print("No f1-score file found.")
        compute_f1(dir)

# Metrics to use for analysis
metrics = ["accuracy", "precision", "recall", "f1-score"]

# Compute mean and confidence intervals for each metric and each model
mean_dict, inter_dict = compute_stats(models, dir_dict, metrics)

# Plot model comparison
plot_frozen_layers(models, metrics, dir_dict, mean_dict, inter_dict, sub_dir, title, save_plot)

## 2. Comparison between 6 models across datasets

In [ ]:
NETWORK = "Mobile"
models = 6
MODEL_1_DIRECTORY = f"VV_VH_WS/results/categorization/{NETWORK}_Numeric-F_BatchSize-8_lr-0.0001_Epochs-20_Folds-5_Norm-T_Aug-T/csv"
MODEL_2_DIRECTORY = f"VV_VH_VH/results/categorization/{NETWORK}_Numeric-F_BatchSize-8_lr-0.0001_Epochs-20_Folds-5_Norm-T_Aug-T/csv"
MODEL_3_DIRECTORY = f"WS_WS_WS/results/categorization/{NETWORK}_Numeric-F_BatchSize-8_lr-0.0001_Epochs-20_Folds-5_Norm-T_Aug-T/csv"
MODEL_4_DIRECTORY = f"WS_sWSO_cWSO/results/categorization/{NETWORK}_Numeric-F_BatchSize-8_lr-0.0001_Epochs-20_Folds-5_Norm-T_Aug-T/csv"
MODEL_5_DIRECTORY = f"VV_VV_VV/results/categorization/{NETWORK}_Numeric-F_BatchSize-8_700x400_lr-0.0001_Epochs-20_Folds-5_Norm-T_Aug-T/csv"
MODEL_6_DIRECTORY = f"VH_VH_VH/results/categorization/{NETWORK}_Numeric-F_BatchSize-8_700x400_lr-0.0001_Epochs-20_Folds-5_Norm-T_Aug-T/csv"

# Critical t-value to see the difference in two distribution with 5 samples
criticalTvalue = 1.533   # 80% confidence
#criticalTvalue = 2.132  # 90% confidence
#criticalTvalue = 2.776  # 95% confidence

In [ ]:
# ACCURACY analysis
model1_accuracy = pickle.load(open(MODEL_1_DIRECTORY + "/test_accuracy.pkl",  "rb" ))
model2_accuracy = pickle.load(open(MODEL_2_DIRECTORY + "/test_accuracy.pkl",  "rb" ))
model3_accuracy = pickle.load(open(MODEL_3_DIRECTORY + "/test_accuracy.pkl",  "rb" ))
model4_accuracy = pickle.load(open(MODEL_4_DIRECTORY + "/test_accuracy.pkl",  "rb" ))
model5_accuracy = pickle.load(open(MODEL_5_DIRECTORY + "/test_accuracy.pkl",  "rb" ))
model6_accuracy = pickle.load(open(MODEL_6_DIRECTORY + "/test_accuracy.pkl",  "rb" ))

model1_accuracy_mean, b_model1_acc, u_model1_acc = mean_confidence_interval(model1_accuracy)
model2_accuracy_mean, b_model2_acc, u_model2_acc = mean_confidence_interval(model2_accuracy)
model3_accuracy_mean, b_model3_acc, u_model3_acc = mean_confidence_interval(model3_accuracy)
model4_accuracy_mean, b_model4_acc, u_model4_acc = mean_confidence_interval(model4_accuracy)
model5_accuracy_mean, b_model5_acc, u_model5_acc = mean_confidence_interval(model5_accuracy)
model6_accuracy_mean, b_model6_acc, u_model6_acc = mean_confidence_interval(model6_accuracy)

acc_dict = {"model1_accuracy": model1_accuracy, "model2_accuracy": model2_accuracy, "model3_accuracy": model3_accuracy,
            "model4_accuracy": model4_accuracy, "model5_accuracy": model5_accuracy, "model6_accuracy": model6_accuracy}
print("---------------- ACCURACY analysis ----------------")
for x, y in itertools.combinations(acc_dict.keys(), 2):
    t, p = scipy.stats.ttest_ind(acc_dict[x], acc_dict[y])
    print("t value accuracy between {} and {}: {}".format(x, y, t))
    if t > criticalTvalue or t < -criticalTvalue :
        print("Null hypothesis rejected -> there is a significant difference in the two accuracy distributions")
    else:
        print("Null hypothesis not rejected -> I assume that each difference in accuracy is due to the randomness")
print("-------------------------------------------------")


# PRECISION analysis
model1_precision = pickle.load(open(MODEL_1_DIRECTORY + "/test_precision.pkl",  "rb" ))
model2_precision = pickle.load(open(MODEL_2_DIRECTORY + "/test_precision.pkl",  "rb" ))
model3_precision = pickle.load(open(MODEL_3_DIRECTORY + "/test_precision.pkl",  "rb" ))
model4_precision = pickle.load(open(MODEL_4_DIRECTORY + "/test_precision.pkl",  "rb" ))
model5_precision = pickle.load(open(MODEL_5_DIRECTORY + "/test_precision.pkl",  "rb" ))
model6_precision = pickle.load(open(MODEL_6_DIRECTORY + "/test_precision.pkl",  "rb" ))

model1_precision_mean, b_model1_p, u_model1_p = mean_confidence_interval(model1_precision)
model2_precision_mean, b_model2_p, u_model2_p = mean_confidence_interval(model2_precision)
model3_precision_mean, b_model3_p, u_model3_p = mean_confidence_interval(model3_precision)
model4_precision_mean, b_model4_p, u_model4_p = mean_confidence_interval(model4_precision)
model5_precision_mean, b_model5_p, u_model5_p = mean_confidence_interval(model5_precision)
model6_precision_mean, b_model6_p, u_model6_p = mean_confidence_interval(model6_precision)

prec_dict = {"model1_precision": model1_precision, "model2_precision": model2_precision, "model3_precision": model3_precision,
            "model4_precision": model4_precision, "model5_precision": model5_precision, "model6_precision": model6_precision}
print("---------------- PRECISION analysis ----------------")
for x, y in itertools.combinations(prec_dict.keys(), 2):
    t, p = scipy.stats.ttest_ind(prec_dict[x], prec_dict[y])
    print("t value precision between {} and {}: {}".format(x, y, t))
    if t > criticalTvalue or t < -criticalTvalue :
        print("Null hypothesis rejected -> there is a significant difference in the two precision distributions")
    else:
        print("Null hypothesis not rejected -> I assume that each difference in precision is due to the randomness")
print("-------------------------------------------------")


# RECALL analysis
model1_recall = pickle.load(open(MODEL_1_DIRECTORY + "/test_recall.pkl",  "rb" ))
model2_recall = pickle.load(open(MODEL_2_DIRECTORY + "/test_recall.pkl",  "rb" ))
model3_recall = pickle.load(open(MODEL_3_DIRECTORY + "/test_recall.pkl",  "rb" ))
model4_recall = pickle.load(open(MODEL_4_DIRECTORY + "/test_recall.pkl",  "rb" ))
model5_recall = pickle.load(open(MODEL_5_DIRECTORY + "/test_recall.pkl",  "rb" ))
model6_recall = pickle.load(open(MODEL_6_DIRECTORY + "/test_recall.pkl",  "rb" ))

model1_recall_mean, b_model1_r, u_model1_r = mean_confidence_interval(model1_recall)
model2_recall_mean, b_model2_r, u_model2_r = mean_confidence_interval(model2_recall)
model3_recall_mean, b_model3_r, u_model3_r = mean_confidence_interval(model3_recall)
model4_recall_mean, b_model4_r, u_model4_r = mean_confidence_interval(model4_recall)
model5_recall_mean, b_model5_r, u_model5_r = mean_confidence_interval(model5_recall)
model6_recall_mean, b_model6_r, u_model6_r = mean_confidence_interval(model6_recall)

rec_dict = {"model1_recall": model1_recall, "model2_recall": model2_recall, "model3_recall": model3_recall,
            "model4_recall": model4_recall, "model5_recall": model5_recall, "model6_recall": model6_recall}
print("---------------- RECALL analysis ----------------")
for x, y in itertools.combinations(rec_dict.keys(), 2):
    t, p = scipy.stats.ttest_ind(rec_dict[x], rec_dict[y])
    print("t value recall between {} and {}: {}".format(x, y, t))
    if t > criticalTvalue or t < -criticalTvalue :
        print("Null hypothesis rejected -> there is a significant difference in the two recall distributions")
    else:
        print("Null hypothesis not rejected -> I assume that each difference in recall is due to the randomness")
print("-------------------------------------------------")


# Compute confidence intervals
confidence_model1 = [u_model1_acc - b_model1_acc, u_model1_p - b_model1_p, u_model1_r - b_model1_r]
confidence_model2 = [u_model2_acc - b_model2_acc, u_model2_p - b_model2_p, u_model2_r - b_model2_r]
confidence_model3 = [u_model3_acc - b_model3_acc, u_model3_p - b_model3_p, u_model3_r - b_model3_r]
confidence_model4 = [u_model4_acc - b_model4_acc, u_model4_p - b_model4_p, u_model4_r - b_model4_r]
confidence_model5 = [u_model5_acc - b_model5_acc, u_model5_p - b_model5_p, u_model5_r - b_model5_r]
confidence_model6 = [u_model6_acc - b_model6_acc, u_model6_p - b_model6_p, u_model6_r - b_model6_r]

# Data to plot
n_groups = 3
means_model1 = (model1_accuracy_mean, model1_precision_mean, model1_recall_mean)
means_model2 = (model2_accuracy_mean, model2_precision_mean, model2_recall_mean)
means_model3 = (model3_accuracy_mean, model3_precision_mean, model3_recall_mean)
means_model4 = (model4_accuracy_mean, model4_precision_mean, model4_recall_mean)
means_model5 = (model5_accuracy_mean, model5_precision_mean, model5_recall_mean)
means_model6 = (model6_accuracy_mean, model6_precision_mean, model6_recall_mean)

# Create plot
fig, ax = plt.subplots(figsize=(15,8))
index = np.arange(n_groups)
bar_width = 0.1
opacity = 0.8

rects1 = plt.bar(index, means_model1, bar_width, yerr = confidence_model1, capsize = 5, alpha = opacity, color = 'g',
                 #label = MODEL_1_DIRECTORY.split(sep='/')[0]
                 label = "Dataset VV+VH+WS")

rects2 = plt.bar(index + bar_width, means_model2, bar_width, yerr = confidence_model2, capsize = 5, alpha = opacity, color = 'r',
                 #label = MODEL_2_DIRECTORY.split(sep='/')[0]
                 label = "Dataset VV+VH+VH")

rects3 = plt.bar(index + 2 * bar_width, means_model3, bar_width, yerr = confidence_model3, capsize = 5, alpha = opacity, color = 'b',
                 #label = MODEL_3_DIRECTORY.split(sep='/')[0]
                 label = "Dataset WS+WS+WS")

rects4 = plt.bar(index + 3 * bar_width, means_model4, bar_width, yerr = confidence_model4, capsize = 5, alpha = opacity, color = 'y',
                 #label = MODEL_4_DIRECTORY.split(sep='/')[0]
                 label = "Dataset WS+sWSO+cWSO")

rects5 = plt.bar(index + 4 * bar_width, means_model5, bar_width, yerr = confidence_model5, capsize = 5, alpha = opacity, color = 'c',
                 #label = MODEL_5_DIRECTORY.split(sep='/')[0]
                 label = "Dataset VV+VV+VV")

rects6 = plt.bar(index + 5 * bar_width, means_model6, bar_width, yerr = confidence_model6, capsize = 5, alpha = opacity, color = 'm',
                 #label = MODEL_6_DIRECTORY.split(sep='/')[0]
                 label = "Dataset VH+VH+VH")

font_size = 15
font_size_ticks = 15
plt.ylabel('Scores', fontsize = font_size)
plt.title(MODEL_1_DIRECTORY.split(sep='/')[-2], fontsize = font_size)
#plt.title("{} across datasets".format(NETWORK), fontsize = font_size)
plt.xticks(index + 1.5*bar_width, ('Accuracy', 'Precision', 'Recall'), fontsize = font_size_ticks)
plt.yticks(fontsize = font_size_ticks)
plt.legend(loc = 'lower left', prop={'size': font_size})
plt.grid(True)
#ax.set_ylim(ymin=0.75)
plt.tight_layout()
plt.show()

# Save plot
save_plot = True
if save_plot:
    dir = "model_comparisons_between_datasets/categorization/"
    os.makedirs(dir, exist_ok=True)
    fig.savefig(dir + "{}_across_{}_datasets.jpg".format(MODEL_1_DIRECTORY.split(sep='/')[-2], models), bbox_inches='tight')
    #fig.savefig(dir + "{}_across_{}_datasets.jpg".format(NETWORK, models), bbox_inches='tight')